# 實驗：超過3k的先分段做摘要，把第一次摘要串起來後，做情節摘要。評估壓成3k以下的事實摘要和情節摘要。

In [1]:
import json

with open('dataset/train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

In [2]:
import requests

url = "https://nlplab-llm.nlpnchu.org/generate"

headers = {
  'Content-Type': 'application/json',
  'accept': 'application/json'
}

def get_breeze(input_content):
    system = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

    payload = json.dumps({
        "inputs": f"<s> {system} [INST] {input_content} [/INST]",
        "parameters": {
            "do_sample": True,
            "temperature": 0.01,
            "top_p": 0.95,
            # "frequency_penalty": 0.01,
            # "presence_penalty": 0.05,
            'repetition_penalty': 1, # 不要一直生重複的
            'max_new_tokens':1000, # 要設一下，不然會講到一半就停掉
        }
        })
    headers = {
    'Content-Type': 'application/json',
    'accept': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, data=payload)
        j_result=json.loads(response.text)
        if "generated_text" in j_result.keys():
            return j_result['generated_text']
        else:
            return j_result
    except Exception as e:
        print(e)

In [3]:
def generate_scenario(judgement):
    # 有範例的情節摘要
    example = '以下是一個情節摘要的範例，輸入是判決書內容，輸出是情節摘要。範例如下：輸入:孫桂蘭與孟奇於民國83年7月10日結婚,孫桂蘭係孟奇之子女即孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉之繼母。孟奇於96年1月22日死亡,孫桂蘭明知孟奇死亡後,其名下財產均為遺產,於遺產分割前屬孫桂蘭、孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉全體繼承人公同共有,未經全體繼承人同意,任一繼承人不得擅自處分,竟意圖為自己不法所有,基於詐欺取財及行使偽造私文書之犯意,於附表各編號所示之時間,至附表各編號所示金融機構,冒用孟奇之名義,在空白取款憑證填寫如附表各編號所示之日期、帳號及金額,並於「取款印鑑」欄,使用孟奇之印章盜蓋印文1枚,表示係孟奇欲自附表各編號所示金融機構帳戶內提領附表各編號所示提款金額,而偽造孟奇名義之取款憑證私文書後,再連同上開帳戶存摺持之向不知情之附表各編號所示金融機構承辦人員辦理取款而行使之,致該金融機構承辦人員陷於錯誤,誤認孫桂蘭係經孟奇本人授權而為上開手續,如數交付如附表各編號所示金額,均足以生損害於孟慶明、孟慶虹、孟慶經、孟慶緯(已歿)、孟蓉之權益及日盛國際商業銀行股份有限公司《下稱日盛商業銀行》、合作金庫商業銀行股份有限公司竹塹分行《下稱合作金庫商業銀行竹塹分行》、中華郵政股份有限公司清華大學郵局《下稱清華大學郵局》、永豐商業銀行股份有限公司《下稱永豐商業銀行》等金融機構對於孟奇帳戶資料、存款管理之正確性。 輸出:在這個案件中,涉及到一段婚姻關係,並有繼承人之間的財產分割。一名已故的個人的遺產成為了糾紛的焦點。在其去世後,其配偶及子女成為了繼承人。然而,配偶明知其去世,卻企圖以詐欺和偽造私文書的方式來獲取他們的共同財產。她冒用已故人的名義,填寫了取款憑證並偽造了他的印章,使金融機構誤以為是合法的取款要求。最終,金融機構執行了這些虛偽的指示,導致繼承人之間的權益受損,也對涉及的金融機構造成了損害。这些行為涉及偽造私文書、詐欺取財,並對繼承人和金融機構產生了不良影響。請學習以上範例，將以下判決書內容做情節摘要，'
    scenario_with_example_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。{example}判決書內容：輸入：{judgement}。輸出：'
    example_scenario = get_breeze(scenario_with_example_prompt)
    example_scenario = example_scenario.replace('\n', '').replace('\t', '')

    # 沒範例的情節摘要
    scenario_without_example_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。判決書內容如下：{judgement}。'
    scenario = get_breeze(scenario_without_example_prompt)
    scenario = scenario.replace('\n', '').replace('\t', '')

    return example_scenario, scenario

In [4]:
def split_passage(judgement):
    summaries = []

    judgement_len = len(judgement)
    window_size = 3000
    overlap_size = 100
    start = 0
    end = min(window_size, judgement_len)

    while start < judgement_len:
        passage = judgement[start:end]
        fact_summary_prompt = f'我會給你一篇法律判決書內容，幫我做事實摘要，可能包含事實情節描述。判決書內容如下：{passage}。'
        summary = get_breeze(fact_summary_prompt)
        summaries.append(summary)
        
        start += window_size - overlap_size
        end = min(start + window_size, judgement_len)

    summary_str = ''.join(summaries)

    return summary_str

In [5]:
def process_judgement(judgement):
    print(f'Origin Length: {len(judgement)}')
    if len(judgement) > 3000:
        print(f'Judgement processing...')
        fact_summary = split_passage(judgement)
        print(f'First summary length: {len(fact_summary)}')
        print(f'First summary complete!')
        print(f'Scenario generating...')
        scenario_with_example, scenario_without_example = generate_scenario(fact_summary)
        print(f'Scenario w example Length: {len(scenario_with_example)}')
        print(f'Scenario w/o example Length: {len(scenario_without_example)}')
        print(f'Scenario complete!')
        print(f'='*100)
        return {
            "judgement": judgement,
            "fact_summary": fact_summary,
            "with_example": scenario_with_example,
            "without_example": scenario_without_example
        }
    else:
        print(f'Scenario generating...')
        scenario_with_example, scenario_without_example = generate_scenario(judgement)
        print(f'Scenario w example Length: {len(scenario_with_example)}')
        print(f'Scenario w/o example Length: {len(scenario_without_example)}')
        print(f'Scenario complete!')
        print(f'='*100)
        return {
            "judgement": judgement,
            "fact_summary": "None",
            "with_example": scenario_with_example,
            "without_example": scenario_without_example
        }

In [6]:
processed_data = []

for data in train_data[:3]:
    judgement = data['fact']
    processed_data.append(process_judgement(judgement))

with open("summary_output/processed_test.json", "w", encoding="utf-8") as json_file:
    json.dump(processed_data, json_file, ensure_ascii=False, indent=4)

Origin Length: 6546
Judgement processing...
First summary length: 905
First summary complete!
Scenario generating...
Scenario w example Length: 255
Scenario w/o example Length: 317
Scenario complete!
Origin Length: 3385
Judgement processing...
First summary length: 758
First summary complete!
Scenario generating...
Scenario w example Length: 403
Scenario w/o example Length: 717
Scenario complete!
Origin Length: 1331
Scenario generating...
Scenario w example Length: 187
Scenario w/o example Length: 367
Scenario complete!
